# TWM Praktikum 5
#### Wintersemester 2019/2020 (Prof. Dr. Markus Döhring, Dr. Steffen Schnitzer, Nina Krüger)

%%bash
# NUR bei RosettaHub ändern: Cell>CellType>Code
for K in $(apt-key list | grep expired | cut -d'/' -f2 | cut -d' ' -f1); do sudo apt-key adv --recv-keys --keyserver keys.gnupg.net $K; done
sudo cat > /etc/apt/sources.list << EOF
deb http://archive.ubuntu.com/ubuntu xenial main restricted universe multiverse
deb-src http://archive.ubuntu.com/ubuntu xenial main restricted universe multiverse
deb http://archive.ubuntu.com/ubuntu xenial-updates main restricted universe multiverse
deb-src http://archive.ubuntu.com/ubuntu xenial-updates main restricted universe multiverse
deb http://archive.ubuntu.com/ubuntu xenial-backports main restricted universe multiverse
deb-src http://archive.ubuntu.com/ubuntu xenial-backports main restricted universe multiverse
deb http://archive.ubuntu.com/ubuntu xenial-security main restricted universe multiverse
deb-src http://archive.ubuntu.com/ubuntu xenial-security main restricted universe multiverse
#deb http://archive.ubuntu.com/ubuntu xenial-proposed restricted main universe multiverse
#deb-src http://archive.ubuntu.com/ubuntu xenial-proposed restricted main universe multiverse
deb http://archive.canonical.com/ubuntu xenial partner
deb-src http://archive.canonical.com/ubuntu xenial partner
EOF
sudo apt-get -y install graphviz

In [ ]:
import sys
if 1==0: #ändern in 1==1, falls auf RosettaHub ausgeführt
    !{sys.executable} -m pip install --upgrade tensorflow-gpu
    !{sys.executable} -m pip install  nltk
    !{sys.executable} -m pip install  sklearn
    !{sys.executable} -m pip install pydotplus
    !{sys.executable} -m pip install  graphviz
    !{sys.executable} -m pip install  --upgrade keras
    !{sys.executable} -m pip install --upgrade tensorflow

In [ ]:
import sys
if 1==0: #ändern in 1==1, falls auf TWM ausgeführt
    !{sys.executable} -m pip install --ignore-installed --upgrade tensorflow-gpu
    !{sys.executable} -m pip install --ignore-installed nltk
    !{sys.executable} -m pip install --ignore-installed sklearn
    !{sys.executable} -m pip install --ignore-installed pydotplus
    !{sys.executable} -m pip install --ignore-installed graphviz
    !{sys.executable} -m pip install --ignore-installed --upgrade keras
    !{sys.executable} -m pip install --ignore-installed --upgrade tensorflow
    !{sys.executable} -m pip uninstall -y numpy
    !{sys.executable} -m pip uninstall -y numpy
    !{sys.executable} -m pip uninstall -y numpy
    !{sys.executable} -m pip install --upgrade numpy

In [ ]:
#imports und keras/tensorflow Einstellungen für Reproduzierbarkeit
import numpy as np
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(2)
tf.set_random_seed(3)

import keras
keras.__version__
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.externals.six import StringIO  
from IPython.display import Image, display  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from keras.datasets import imdb

import json
import numpy
import nltk

from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import imdb
from keras.preprocessing import sequence

from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
#wir setzen eine globalen Parameter für die maximale Länge von Reviews.
#Alle Reviews, die länger als review_length Wörter sind, werden vor der Modellbildung entsprechend gekürzt 
review_length = 500

#ein weiterer globaler Parameter gibt an, wie groß das Dictionnary sein darf.
#Es werden grundsätzlich die nwords am häufigsten im Corpus vorkommenden Wörter verwendet. 
nwords = 10000


Hinweis: ein Großteil dieses Notebooks basiert auf dem Buch "Deep Learning with Python" (Chollet 2017), insbesondere Kapitel 6, und entsprechend bereitgestelltem Code: https://github.com/fchollet/deep-learning-with-python-notebooks


## 1. IMDB Movie Reviews - Explorative Datenanalyse

In Praktikum 5 arbeiten wir mit dem IMDB Datensatz, der durch Keras bereitgestellt wird. Dieser enthält insgesamt 50.000 stark positiv/negativ geprägte Reviews. Der folgende Python Code liefert 25.000 Reviews (X\_ Arrays) mit entsprechenden Labels (Y\_ Arrays) für das Training und 25.000 Reviews für das Testen eines Modells. 

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=nwords)

Die von Keras bereitgestellten Datenstrukturen bestehen aus Arrays von Ganzzahlen, die Einträge in einem Lexikon darstellen. Um uns Texte anschauen (oder ggf. textuell weiterverarbeiten) zu können, müssen wir diese "zurückübersetzen", d.h. die Ganzzahlen wieder in Wörter transformieren. Weiterhin kürzen wir die Reviews auf eine in den globelen Parmetern gesetzte Länge. 

In [ ]:
# word_index ist ein dictionnary, das Wörter auf eine Ganzzahl abbildet
word_index = imdb.get_word_index()
# wir kehren den Word Index um und bilden nun Ganzzahlen auf Wörter ab
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Wir dekodieren die Reviews. Achtung: Die Indizes sind um 3 verschoben
# weil 0, 1 and 2 reservierte Indizes für "padding", "start of sequence" und "unknown" sind
X_train = [sequence[0:min(review_length,len(sequence))] for sequence in X_train]
X_test = [sequence[0:min(review_length,len(sequence))] for sequence in X_test]
X_train = [
    ' '.join(
        reverse_word_index.get(i - 3, '?')
        for i in X_train[i]
        if i < len(reverse_word_index)
    ) for i in range(len(X_train))
]

X_test = [
    ' '.join(
        reverse_word_index.get(i - 3, '?')
        for i in X_test[i]
        if i < len(reverse_word_index)
    ) for i in range(len(X_test))
]

### Aufgabe 1: Schauen Sie sich einige Reviews und deren Labels an
Ist ein Datensatz mit 1 gelabelt, dann steht dies fuer einen eher positives Review. Die 0 als Label steht fuer ein eher negatives Review.

In [ ]:
#IHR CODE HIER

### Aufgabe 2: Identifizieren Sie die relevantesten Features (Wörter) anhand des Chi²-Wertes

In [ ]:
#IHR CODE HIER

### Aufgabe 3: Prüfen und begründen Sie, ob „accuracy“ ein sinnvolles Gütemaß für einen Klassifikator auf diesem Datensatz ist.  

In [ ]:
#IHR CODE+BEGRÜNDUNG HIER

## 2. Deep Learning
### 2.1. RNN mit Embeddings


Im folgenden lernen wir ein RNN mit Embeddings Layer über 10 Iterationen (Epochen). Für jede Epoche wird die Performance (accuracy) für die Trainings- und die Validierungsdaten (20%, die vom ursprünglichen Trainingsdatensatz nochmal beiseite gelegt wurdden) ausgegeben. Am Ende werden die entsprechenden Werte im Zeitverlauf graphisch dargestellt. 

In [ ]:
from keras.preprocessing import sequence


print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=nwords)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=review_length)
input_test = sequence.pad_sequences(input_test, maxlen=review_length)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

In [ ]:
%%time
from keras.layers import Dense
import keras 
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/home/student/PycharmProjects/test/logs/', histogram_freq=0, write_graph=True, write_images=True)
model = Sequential()
model.add(Embedding(nwords, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2#,
                     #callbacks=[tbCallBack]
                   )            


In [ ]:
def plothist(hist):
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()


    plt.show()
    

In [ ]:
plothist(history)


### Aufgabe 4: Wie interpretieren Sie die Performance-Kurven zum RNN bzw. was fällt Ihnen auf?

IHRE INTERPRETATION HIER

Basierend auf der Train-Validation Performance entscheiden wir, unser Modell auf den gesamten Trainingsdaten über 4 Epochen zu trainieren. Dann geben wir die Performance auf den Testdaten aus. 

In [ ]:
model.reset_states()

model.fit(input_train, y_train, epochs=2, batch_size=128)
results = model.evaluate(input_test, y_test)
results[1]



### Aufgabe 5: Schauen Sie sich einige der "drastischsten" false positives und false negatives an (hohe Score und Label=1 oder niedrige Score und Label=0). Können Sie erahnen, was das Modell ggf. verwirrt hat?

In [ ]:
#IHR CODE+BEGRÜNDUNG HIER

### 2.2 RNN mit Glove 50 Layer

Nun trainieren wir ein RNN mit der selben Architektur wie vorher, nur dass wir anstelle eines parallel trainierten 32-dimensionalen Embeddings ein 50-dimensionales, vortrainiertes Word-Embedding verwenden.

In [ ]:
import os
import numpy as np
from io import BytesIO
from zipfile import ZipFile
import requests

embeddings_index = {}

glove6B = 'glove.6B.50d.txt'

if os.path.isfile(glove6B):
    print('Glove tokens file already exists. No need to download it.')
    f = open(glove6B)
else:
    print('Glove tokens file does not exists. Download can take some time.')
    url = 'http://nlp.stanford.edu/data/glove.6B.zip'
    resp = requests.get(url)
    zipfile = ZipFile(BytesIO(resp.content))
    f = zipfile.open(glove6B)
    #saving it for the next run
    zipfile.extract(glove6B)

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

from keras.datasets import imdb
embedding_dim = 50
word_index = imdb.get_word_index()
embedding_matrix = np.zeros((nwords, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < nwords:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
%%time
model = Sequential()
model.add(Embedding(nwords, 50))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2#,
                     #callbacks=[tbCallBack]
                   )            


In [ ]:
plothist(history)


In [ ]:
model.reset_states()
model.fit(input_train, y_train, epochs=3, batch_size=128)
results = model.evaluate(input_test, y_test)
results

### Aufgabe 6: Optional (nur wenn Sie gut in der Zeit liegen und fit in der Materie sind!): Wie interpretieren Sie die Performance-Kurven und Ergebnisse auf den Testdaten zum RNN mit vorgelernten Glove-Embedding bzw. was fällt Ihnen auf

IHRE INTERPRETATION HIER

### 2.3 LSTM

Nun trainieren wir anstatt eines Modells mit normalen RNN Zellen ein Modell mit LSTM Zellen. Ansonsten bleiben alle anderen Rahmenbedingungen identisch zum Modell unter 2.1

In [ ]:
%%time 
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(nwords, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

In [ ]:
plothist(history)

In [ ]:
model.reset_states()
model.fit(input_train, y_train, epochs=4, batch_size=128)
results = model.evaluate(input_test, y_test)
results

### Aufgabe 7: Wie interpretieren Sie die Performance-Kurven und Ergebnisse auf den Testdaten zum LSTM bzw. was fällt Ihnen auf?

IHRE INTERPRETATION HIER

## 3. Not So Deep Learning

Im folgenden verwenden wir sehr einfache Klassifikationsmodelle: Eine logistische Regression (siehe ggf. Wikipedia o.ä.) und einen Decision Tree aufbauend auf TF-IDF gewichteten Unigrammen+Bigrammen. 

## Logistische Regression

In [ ]:
%%time
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('log', LogisticRegression())
])

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))
print((y_pred == y_test).mean())

In [ ]:
# Ausgabe der Features sortiert nach ihren Koeffizienten innerhalb des LogReg Modells
np.array(model.named_steps['tfidf'].get_feature_names())[np.argsort(model.named_steps['log'].coef_[0])]

## Decision Tree

In [ ]:
%%time
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('tree', DecisionTreeClassifier(criterion='gini', max_depth=4))
])

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))
print((y_pred == y_test).mean())

In [ ]:
dot_data = StringIO()

export_graphviz(model.named_steps['tree'], out_file=dot_data, filled=True, rounded=True, special_characters=True)

splitArgs = model.named_steps['tree'].tree_.feature.tolist()

dot_data = dot_data.getvalue()

print('Das sind die Splitargumente des DecisionTrees:')

for x in ([arg for arg in splitArgs if arg != -2 ]):
    rep = model.named_steps['tfidf'].get_feature_names()[x] 
    dot_data = dot_data.replace('X<SUB>'+str(x)+'</SUB>', str(rep))
    print(rep)

In [ ]:
%matplotlib notebook
# Falls Sie https://www.graphviz.org/ installiert haben, koennen Sie den Decision Tree auch plotten
# In der twm VM koennen Sie graphviz mit: sudo apt-get -y install graphviz
# installieren
try:
    graph = pydotplus.graph_from_dot_data(dot_data)
    display(Image(graph.create_png()))
except:
    print('graphviz ist nicht installiert')

### Aufgabe 8: Wie interpretieren Sie die Ergebnisse der "einfacheren" Klassifikationsmodelle auf den Testdaten bzw. was fällt Ihnen auf? Inwiefern deckt sich die Wichtigkeit der Features mit der, die Sie in Aufgabe (2) ermittelt haben?

IHRE INTERPRETATION HIER